In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print("imported")
import random
import os
import sys
import itertools
sys.path.append('src/')
import nn
import process_data
import nibabel as nib
from math import floor, ceil
# import cv2

from sklearn.metrics import confusion_matrix
import scipy.sparse
from scipy.misc import imrotate, imresize
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import rotate
from skimage import exposure
from skimage.io import imread, imsave
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
print(local_device_protos)

import pipeline
import Unet

imported
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14326521798892636034
]


In [4]:
test_trial_dir = "/Users/nozik/Documents/HARTresearch/allpreprocessed/trial8_30_fs"
raw, seg = pipeline.load_data(test_trial_dir)

====
/Users/nozik/Documents/HARTresearch/allpreprocessed/trial8_30_fs
====


NameError: name 'os' is not defined